In [1]:
!pip install transformers torch langchain pandas 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.0/368.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 311.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 79.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 369.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uni

In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 94.3 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 118.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 130.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.4.1
    Uninstalling safetensors-0.4.1:
      Successfully uninstalled safetensors-0.4.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.2
    Uninstalling huggingface-hub-0.20.2:
      Successfully uninstalled huggingface-hub-0.20.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
  

In [3]:
from huggingface_hub import login
login(token="")

In [5]:
import os
import re
import unicodedata
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import pipeline

# --------------------------
# Archivos
# --------------------------
archivo_a = "terminos_inst_1shot_limpio.csv"
archivo_b = "eurovoc_inst_3shot_limpio.csv"
gold_csv = "gold_standard_enlazado.csv"
salida_csv = "enlaces_detectados_one_shot.csv"
respuestas_completas_csv = "respuestas_completas_one_shot.csv"
matches_gold_csv = "matches_en_gold_standard_one_shot.csv"

# --------------------------
# Cargar y normalizar CSVs
# --------------------------
df_a = pd.read_csv(archivo_a)
df_b = pd.read_csv(archivo_b)
gold_df = pd.read_csv(gold_csv)

for df in (df_a, df_b, gold_df):
    df.columns = df.columns.str.strip().str.lower()

df_a = df_a.rename(columns={"término": "termino", "ventana_de_contexto": "ventana_de_contexto"})
df_b = df_b.rename(columns={"término": "termino", "ventana_de_contexto": "ventana_de_contexto"})
gold_df = gold_df.rename(columns={"término_1": "termino_1", "término_2": "termino_2"})

# --------------------------
# Inicializar modelo
# --------------------------
modelo = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.1-8B-Instruct",
    device_map="auto"
)

# --------------------------
# Función para extraer 'sí' o 'no'
# --------------------------
def extraer_si_o_no(respuesta: str) -> str:
    texto = unicodedata.normalize("NFD", respuesta.lower())
    texto = "".join(ch for ch in texto if unicodedata.category(ch) != "Mn")
    for linea in texto.splitlines():
        if re.match(r"^\s*(si|sí)\b", linea):
            return "si"
        elif re.match(r"^\s*no\b", linea):
            return "no"
    m = re.search(r'\b(si|sí|no)\b', texto)
    return "si" if m and m.group(1).startswith("s") else "no" if m and m.group(1) == "no" else "desconocido"

# --------------------------
# Prompt one-shot (ejemplo incluido)
# --------------------------
ejemplo = (
    "Término 1: salario mínimo\n"
    "Contexto 1: El salario mínimo es la remuneración mínima legal que un trabajador debe recibir por su labor.\n\n"
    "Término 2: remuneración básica\n"
    "Contexto 2: La remuneración básica establece el ingreso mínimo que puede recibir un trabajador formal.\n\n"
    "Respuesta: Sí, ambos términos hacen referencia al ingreso mínimo legal, por lo tanto representan el mismo concepto.\n\n"
)

# --------------------------
# Evaluación con one-shot prompting
# --------------------------
def evaluar_similitud(t1, ctx1, t2, ctx2):
    prompt = (
        f"<s>[INST] Eres un experto en lingüística jurídica. "
        f"Determina si los siguientes términos representan el mismo concepto o deben enlazarse.\n\n"
        f"{ejemplo}"
        f"Término 1: {t1}\nContexto 1: {ctx1}\n\n"
        f"Término 2: {t2}\nContexto 2: {ctx2}\n\n"
        f"Respuesta: [/INST]"
    )
    try:
        salida = modelo(prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
        return salida.strip()
    except Exception as e:
        return f"Error: {e}"

# --------------------------
# Comparar todos los pares A × B
# --------------------------
pares_detectados = []
respuestas_completas = []

for _, row_a in df_a.iterrows():
    termino_a = row_a["termino"].strip().lower()
    contexto_a = row_a["ventana_de_contexto"]

    for _, row_b in df_b.iterrows():
        termino_b = row_b["termino"].strip().lower()
        contexto_b = row_b["ventana_de_contexto"]

        respuesta = evaluar_similitud(termino_a, contexto_a, termino_b, contexto_b)
        decision = extraer_si_o_no(respuesta)

        respuestas_completas.append({
            "termino_1": termino_a,
            "termino_2": termino_b,
            "respuesta_completa": respuesta,
            "decision": decision
        })

        if decision == "si":
            pares_detectados.append({
                "termino_1": termino_a,
                "termino_2": termino_b,
                "respuesta_completa": respuesta,
                "decision": decision
            })

# --------------------------
# Guardar resultados
# --------------------------
pd.DataFrame(pares_detectados).to_csv(salida_csv, index=False, encoding="utf-8")
pd.DataFrame(respuestas_completas).to_csv(respuestas_completas_csv, index=False, encoding="utf-8")
print(f"✅ Enlaces detectados guardados en: {salida_csv}")
print(f"📋 Respuestas completas guardadas en: {respuestas_completas_csv}")

print(f"\n🔎 Total respuestas: {len(respuestas_completas)}")
print(f"✅ Enlaces detectados (sí): {len(pares_detectados)}")
print(f"❌ No enlazados o desconocidos: {len(respuestas_completas) - len(pares_detectados)}")

# --------------------------
# Evaluación contra gold standard
# --------------------------
gold_set = set(
    (a.strip().lower(), b.strip().lower())
    for a, b in zip(gold_df["termino_1"], gold_df["termino_2"])
)
detected_set = set((p["termino_1"], p["termino_2"]) for p in pares_detectados)

# Construir y_true y y_pred
y_true = []
y_pred = []
for pair in detected_set:
    y_pred.append(1)
    y_true.append(1 if pair in gold_set else 0)

if y_true:
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print("\n📊 Evaluación contra el gold standard:")
    print(f"🔹 Precision: {precision:.4f}")
    print(f"🔹 Recall:    {recall:.4f}")
    print(f"🔹 F1 Score:  {f1:.4f}")
else:
    print("⚠️ No hay coincidencias detectadas para evaluar.")

# --------------------------
# Guardar aciertos (verdaderos positivos)
# --------------------------
matches_gold = [
    p for p in pares_detectados
    if (p["termino_1"], p["termino_2"]) in gold_set
]
pd.DataFrame(matches_gold).to_csv(matches_gold_csv, index=False, encoding="utf-8")
print(f"✅ Aciertos guardados en: {matches_gold_csv}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VE

✅ Enlaces detectados guardados en: enlaces_detectados_one_shot.csv
📋 Respuestas completas guardadas en: respuestas_completas_one_shot.csv

🔎 Total respuestas: 3969
✅ Enlaces detectados (sí): 3969
❌ No enlazados o desconocidos: 0

📊 Evaluación contra el gold standard:
🔹 Precision: 0.0169
🔹 Recall:    1.0000
🔹 F1 Score:  0.0333
✅ Aciertos guardados en: matches_en_gold_standard_one_shot.csv


In [4]:
import os
import re
import unicodedata
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import pipeline

# --------------------------
# Archivos
# --------------------------
archivo_a = "terminos_inst_1shot_limpio.csv"
archivo_b = "eurovoc_inst_3shot_limpio.csv"
gold_csv = "gold_standard_enlazado.csv"
salida_csv = "enlaces_detectados_two_shot.csv"
respuestas_completas_csv = "respuestas_completas_two_shot.csv"
matches_gold_csv = "matches_en_gold_standard_two_shot.csv"

# --------------------------
# Cargar y normalizar CSVs
# --------------------------
df_a = pd.read_csv(archivo_a)
df_b = pd.read_csv(archivo_b)
gold_df = pd.read_csv(gold_csv)

for df in (df_a, df_b, gold_df):
    df.columns = df.columns.str.strip().str.lower()

df_a = df_a.rename(columns={"término": "termino", "ventana_de_contexto": "ventana_de_contexto"})
df_b = df_b.rename(columns={"término": "termino", "ventana_de_contexto": "ventana_de_contexto"})
gold_df = gold_df.rename(columns={"término_1": "termino_1", "término_2": "termino_2"})

# --------------------------
# Inicializar modelo
# --------------------------
modelo = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.1-8B-Instruct",
    device_map="auto"
)

# --------------------------
# Función mejorada para extraer 'sí' o 'no'
# --------------------------
def extraer_si_o_no(respuesta: str) -> str:
    texto = unicodedata.normalize("NFD", respuesta.lower())
    texto = "".join(ch for ch in texto if unicodedata.category(ch) != "Mn")

    for linea in texto.splitlines():
        if re.match(r"^\s*(si|sí)\b", linea):
            return "si"
        elif re.match(r"^\s*no\b", linea):
            return "no"

    m = re.search(r'\b(si|sí|no)\b', texto)
    return "si" if m and m.group(1).startswith("s") else "no" if m and m.group(1) == "no" else "desconocido"

# --------------------------
# Función de evaluación
# --------------------------
def evaluar_similitud(t1, ctx1, t2, ctx2):
    prompt = (
        "<s>[INST] Eres un experto en lingüística jurídica. "
        "¿Los siguientes dos términos representan exactamente el mismo concepto en el contexto laboral, "
        "de modo que deberían ser enlazados en un sistema de conceptos?\n\n"
        f"Término 1: {t1}\nContexto 1: {ctx1}\n\n"
        f"Término 2: {t2}\nContexto 2: {ctx2}\n\n"
        "Solo enlaza si son *conceptualmente idénticos*. No enlaces si son solo similares o relacionados.\n\n"
        "Responde únicamente con 'Sí' o 'No', seguido de una breve justificación. [/INST]"
    )
    try:
        salida = modelo(prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
        return salida.strip()
    except Exception as e:
        return f"Error: {e}"

# --------------------------
# Comparar todos los pares A × B
# --------------------------
pares_detectados = []
respuestas_completas = []

for _, row_a in df_a.iterrows():
    termino_a = row_a["termino"].strip().lower()
    contexto_a = row_a["ventana_de_contexto"]

    for _, row_b in df_b.iterrows():
        termino_b = row_b["termino"].strip().lower()
        contexto_b = row_b["ventana_de_contexto"]

        respuesta = evaluar_similitud(termino_a, contexto_a, termino_b, contexto_b)
        decision = extraer_si_o_no(respuesta)

        respuestas_completas.append({
            "termino_1": termino_a,
            "termino_2": termino_b,
            "respuesta_completa": respuesta,
            "decision": decision
        })

        if decision == "si":
            pares_detectados.append({
                "termino_1": termino_a,
                "termino_2": termino_b,
                "respuesta_completa": respuesta,
                "decision": decision
            })

# --------------------------
# Añadir ejemplo negativo manual
# --------------------------
ejemplo_negativo = {
    "termino_1": "desempleo juvenil",
    "contexto_1": "Medidas para combatir el desempleo juvenil en Europa.",
    "termino_2": "contrato a término indefinido",
    "contexto_2": "Regulación legal del contrato a término indefinido en el sector privado."
}

respuesta = evaluar_similitud(
    ejemplo_negativo["termino_1"],
    ejemplo_negativo["contexto_1"],
    ejemplo_negativo["termino_2"],
    ejemplo_negativo["contexto_2"]
)
decision = extraer_si_o_no(respuesta)

respuestas_completas.append({
    **ejemplo_negativo,
    "respuesta_completa": respuesta,
    "decision": decision
})

if decision == "si":
    pares_detectados.append({
        **ejemplo_negativo,
        "respuesta_completa": respuesta,
        "decision": decision
    })

# --------------------------
# Guardar resultados
# --------------------------
pd.DataFrame(pares_detectados).to_csv(salida_csv, index=False, encoding="utf-8")
pd.DataFrame(respuestas_completas).to_csv(respuestas_completas_csv, index=False, encoding="utf-8")
print(f"✅ Enlaces detectados guardados en: {salida_csv}")
print(f"📋 Respuestas completas guardadas en: {respuestas_completas_csv}")

print(f"\n🔎 Total respuestas: {len(respuestas_completas)}")
print(f"✅ Enlaces detectados (sí): {len(pares_detectados)}")
print(f"❌ No enlazados o desconocidos: {len(respuestas_completas) - len(pares_detectados)}")

# --------------------------
# Evaluación contra gold standard
# --------------------------
gold_set = set(
    (a.strip().lower(), b.strip().lower())
    for a, b in zip(gold_df["termino_1"], gold_df["termino_2"])
)

# Agregar explícitamente el ejemplo negativo como no enlazado (negativo real)
gold_set.add(("desempleo juvenil", "contrato a término indefinido"))  # etiqueta 0 esperada

detected_set = set((p["termino_1"], p["termino_2"]) for p in pares_detectados)

# Construir y_true y y_pred
y_true = []
y_pred = []

# Crear conjunto completo de todos los pares evaluados (positivos y negativos)
all_evaluated_pairs = set((r["termino_1"], r["termino_2"]) for r in respuestas_completas)

for pair in all_evaluated_pairs:
    y_pred.append(1 if pair in detected_set else 0)
    y_true.append(1 if pair in gold_set else 0)

if y_true:
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print("\n📊 Evaluación contra el gold standard (con ejemplo negativo):")
    print(f"🔹 Precision: {precision:.4f}")
    print(f"🔹 Recall:    {recall:.4f}")
    print(f"🔹 F1 Score:  {f1:.4f}")
else:
    print("⚠️ No hay coincidencias detectadas para evaluar.")

# --------------------------
# Guardar aciertos (verdaderos positivos)
# --------------------------
matches_gold = [
    p for p in pares_detectados
    if (p["termino_1"], p["termino_2"]) in gold_set
]
pd.DataFrame(matches_gold).to_csv(matches_gold_csv, index=False, encoding="utf-8")
print(f"✅ Aciertos guardados en: {matches_gold_csv}")


2025-07-31 11:38:40.247039: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VE

✅ Enlaces detectados guardados en: enlaces_detectados_two_shot.csv
📋 Respuestas completas guardadas en: respuestas_completas_two_shot.csv

🔎 Total respuestas: 3970
✅ Enlaces detectados (sí): 103
❌ No enlazados o desconocidos: 3867

📊 Evaluación contra el gold standard (con ejemplo negativo):
🔹 Precision: 0.4186
🔹 Recall:    0.5625
🔹 F1 Score:  0.4800
✅ Aciertos guardados en: matches_en_gold_standard_two_shot.csv
